### Self-Attention in Transformers

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (self.head_dim * heads == embed_size), "Embed size must be divisible by number of heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        # Number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.view(N, value_len, self.heads, self.head_dim)
        keys = keys.view(N, key_len, self.heads, self.head_dim)
        queries = query.view(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        # Calculate attention scores
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Apply softmax to get attention weights
        attention = torch.nn.functional.softmax(energy / (self.embed_size ** (1/2)), dim=3)

        # Calculate the weighted sum of the values
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

# Example usage
if __name__ == "__main__":
    embed_size = 256
    heads = 8
    seq_len = 10
    batch_size = 32

    # Create random input tensors
    values = torch.rand((batch_size, seq_len, embed_size))
    keys = torch.rand((batch_size, seq_len, embed_size))
    query = torch.rand((batch_size, seq_len, embed_size))
    mask = None  # You can define a mask if needed

    # Initialize the self-attention layer
    self_attention = SelfAttention(embed_size, heads)

    # Forward pass
    output = self_attention(values, keys, query, mask)
    print(output.shape)  # Should be (batch_size, seq_len, embed_size)

### Explanation:
1. **Initialization**:
   - `embed_size`: The size of the embeddings.
   - `heads`: The number of attention heads.
   - `head_dim`: The dimension of each head, which is `embed_size // heads`.

2. **Forward Pass**:
   - The input tensors (`values`, `keys`, `query`) are reshaped to split them into multiple heads.
   - Linear transformations are applied to `values`, `keys`, and `queries`.
   - The attention scores are computed using the dot product of `queries` and `keys`.
   - If a mask is provided, it is applied to the attention scores.
   - The softmax function is applied to the attention scores to get the attention weights.
   - The weighted sum of the `values` is computed using the attention weights.
   - The output is reshaped and passed through a final linear layer to get the final output.